# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.sample(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
103,103,Iquique,-20.2208,-70.1431,25.02,79,3,2.66,CL,1680729434
460,460,Trairi,-3.2778,-39.2689,24.43,91,99,1.54,BR,1680729346
254,254,Alugan,12.2188,125.4808,23.93,82,93,2.74,PH,1680729472
37,37,New Norfolk,-42.7826,147.0587,11.72,73,25,1.11,AU,1680729418
251,251,Raudeberg,61.9875,5.1352,6.04,63,100,8.57,NO,1680729472
485,485,Lethem,3.3803,-59.7968,34.03,52,20,7.20,GY,1680729534
542,542,Tezu,27.9167,96.1667,18.08,76,82,1.55,IN,1680729550
371,371,Tateyama,34.9833,139.8667,17.39,69,100,9.73,JP,1680729505
23,23,Plum,40.5004,-79.7495,30.26,36,91,1.34,US,1680729415
246,246,Rockland,41.1668,-74.0496,10.93,88,100,1.34,US,1680729331


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# %%capture --no-display

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "ESRI",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity")  



# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [3]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 27 degrees but higher than 18, Wind speed less than 4.5 m/s, and  Zero cloudiness

cities_filter_df = city_data_df.loc[(city_data_df["Max Temp"] > 18) & (city_data_df["Max Temp"] < 27) & 
                                    (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
cities_filter_df = cities_filter_df.dropna()

# Display sample data
cities_filter_df.sample(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
270,270,Matay,28.4190,30.7792,19.23,33,0,4.28,EG,1680729476
32,32,Saldanha,-33.0117,17.9442,19.03,62,0,4.11,ZA,1680729417
500,500,Annamalainagar,11.4000,79.7333,26.29,85,0,2.76,IN,1680729538
212,212,Dakar,14.6937,-17.4441,24.49,78,0,3.09,SN,1680729461
393,393,Gilroy,37.0058,-121.5683,18.29,41,0,1.54,US,1680729213


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cities_filter_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel
32,Saldanha,ZA,-33.0117,17.9442,62,
58,Cape Town,ZA,-33.9258,18.4232,71,
76,Yulara,AU,-25.2406,130.9889,63,
212,Dakar,SN,14.6937,-17.4441,78,
270,Matay,EG,28.4190,30.7792,33,
307,Laguna,US,38.4210,-121.4238,39,
334,Sur,OM,22.5667,59.5289,64,
372,Tura,IN,25.5198,90.2201,85,
393,Gilroy,US,37.0058,-121.5683,41,
425,Qeshm,IR,26.9581,56.2719,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [66]:
# Set parameters to search for a hotel
radius = 10000
params = {
          "categories" : "accommodation.hotel"
          , "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get the city's name
    city = row["City"]
    
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
   
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")


# Display sample data
hotel_df


Starting hotel search
Saldanha - nearest hotel: Hoedjesbaai
Cape Town - nearest hotel: Townhouse Hotel
Yulara - nearest hotel: Outback Pioneer Hotel & Lodge
Dakar - nearest hotel: 翁记小吃
Matay - nearest hotel: No hotel found
Laguna - nearest hotel: Holiday Inn Express & Suites
Sur - nearest hotel: Sur Hotel
Tura - nearest hotel: No hotel found
Gilroy - nearest hotel: Days Inn by Wyndham Gilroy
Qeshm - nearest hotel: هتل آپادانا
Annamalainagar - nearest hotel: Hotel Grand Palace Stay
Necochea - nearest hotel: Cabañas Río Quequén
Tahta - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel,Hotel Name
32,Saldanha,ZA,-33.0117,17.9442,62,,Hoedjesbaai
58,Cape Town,ZA,-33.9258,18.4232,71,,Townhouse Hotel
76,Yulara,AU,-25.2406,130.9889,63,,Outback Pioneer Hotel & Lodge
212,Dakar,SN,14.6937,-17.4441,78,,翁记小吃
270,Matay,EG,28.4190,30.7792,33,,No hotel found
307,Laguna,US,38.4210,-121.4238,39,,Holiday Inn Express & Suites
334,Sur,OM,22.5667,59.5289,64,,Sur Hotel
372,Tura,IN,25.5198,90.2201,85,,No hotel found
393,Gilroy,US,37.0058,-121.5683,41,,Days Inn by Wyndham Gilroy
425,Qeshm,IR,26.9581,56.2719,69,,هتل آپادانا


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "ESRI",
    frame_width = 800,
    frame_height = 600,
    )  

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]